<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sco

# サンプルデータの作成
np.random.seed(42)
returns = np.random.randn(100, 4)  # 100日間のリターン、4つの資産

# 年次リターンと共分散行列を計算
mean_returns = returns.mean(axis=0)
cov_matrix = np.cov(returns, rowvar=False)
num_assets = len(mean_returns)

# リスクフリーレートを定義
risk_free_rate = 0.01

# ポートフォリオのパフォーマンスを計算する関数
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, risk

# 最適化の目的関数（シャープレシオの最小化）
def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_returns, p_risk = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_returns - risk_free_rate) / p_risk

# 制約条件と境界条件
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for asset in range(num_assets))

# 初期の重み
initial_weights = num_assets * [1. / num_assets,]

# 最適化の実行
opts = sco.minimize(negative_sharpe_ratio, initial_weights, args=(mean_returns, cov_matrix, risk_free_rate),
                    method='SLSQP', bounds=bounds, constraints=constraints)

# 最適なポートフォリオの重み
optimal_weights = opts.x

# 最適なポートフォリオのパフォーマンス
opt_returns, opt_risk = portfolio_performance(optimal_weights, mean_returns, cov_matrix)
opt_sharpe_ratio = (opt_returns - risk_free_rate) / opt_risk

# 結果の表示
print("最適なポートフォリオの重み:", optimal_weights)
print("最適なポートフォリオのリターン:", opt_returns)
print("最適なポートフォリオのリスク:", opt_risk)
print("最適なポートフォリオのシャープレシオ:", opt_sharpe_ratio)

# ポートフォリオの可視化
def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results = np.zeros((3, num_portfolios))
    weight_array = []
    for i in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        weight_array.append(weights)
        portfolio_return, portfolio_stddev = portfolio_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_return
        results[1,i] = portfolio_stddev
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_stddev

    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[1,max_sharpe_idx], results[0,max_sharpe_idx]
    max_sharpe_allocation = weight_array[max_sharpe_idx]

    min_vol_idx = np.argmin(results[1])
    sdp_min, rp_min = results[1,min_vol_idx], results[0,min_vol_idx]
    min_vol_allocation = weight_array[min_vol_idx]

    plt.figure(figsize=(10, 7))
    plt.scatter(results[1,:],results[0,:],c=results[2,:],cmap='YlGnBu', marker='o')
    plt.scatter(sdp, rp, marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min, rp_min, marker='*',color='g',s=500, label='Minimum volatility')

    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.colorbar(label='Sharpe ratio')
    plt.legend(labelspacing=0.8)

display_simulated_ef_with_random(mean_returns, cov_matrix, 10000, risk_free_rate)
plt.show()
